In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.metrics import r2_score, mean_squared_error
import optuna
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import Pipeline
import joblib
import random

C:\Users\ucbab\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:

data = pd.read_excel("C:/Users/ucbab/OneDrive/Documents/Pasc-Data.xlsx")

In [6]:
x= data.drop(columns=['Biodiesel yield']).values
y=data['Biodiesel yield'].values

In [7]:
scaler_x = StandardScaler()
scaler_y= StandardScaler()
x = scaler_x.fit_transform(x)
y=scaler_y.fit_transform(y.reshape(-1,1))


In [8]:
SEED = 42
np.random.seed(SEED)


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=SEED)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=SEED)


In [7]:
best_score= -np.inf
best_model=None

def objective(trial):
    global best_score

    C = trial.suggest_loguniform("C", 0.01, 1.5)
    epsilon = trial.suggest_loguniform("epsilon", 1e-2, 1.0)
    gamma = trial.suggest_loguniform("gamma", 1e-4, 0.1)
    kernel = trial.suggest_categorical("kernel", ["rbf", 'poly'])

    svr = SVR(C=C, epsilon=epsilon, gamma=gamma, kernel=kernel)

    kf = KFold(n_splits=5, shuffle=True, random_state=SEED)
    scores = cross_val_score(svr, x_train, y_train, cv=kf, scoring="r2")
    mean_score=np.mean(scores)

    if mean_score > best_score:
        best_score = mean_score
        best_model=svr
        svr.fit(x_train,y_train)
        joblib.dump(best_model, 'best_svr_prof_model.pkl')
    return mean_score    

In [18]:
study= optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

[I 2025-05-20 13:02:17,837] A new study created in memory with name: no-name-45482e46-9f2a-4fd4-a27b-e7a289f2f869
C:\Users\ucbab\AppData\Local\Temp\ipykernel_7036\1108954070.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  C = trial.suggest_loguniform("C", 0.01, 1.5)
C:\Users\ucbab\AppData\Local\Temp\ipykernel_7036\1108954070.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  epsilon = trial.suggest_loguniform("epsilon", 1e-2, 1.0)
C:\Users\ucbab\AppData\Local\Temp\ipykernel_7036\1108954070.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use sugge

In [10]:
best_model=joblib.load('best_svr_prof_model.pkl')
y_test_pred = best_model.predict(x_test)
y_train_pred = best_model.predict(x_train)
r2_test=r2_score(y_test,y_test_pred)
r2_train=r2_score(y_train,y_train_pred)

print(f'Train R2: {r2_train:.4f}')
print(f'Test R2: {r2_test:.4f}')

Train R2: 0.9562
Test R2: 0.9061


In [3]:
import sklearn

print(f"SVR model version: {SVR.__module__} {SVR.__version__ if hasattr(SVR, '__version__') else ''}")
print(f"scikit-learn version: {sklearn.__version__}")

SVR model version: sklearn.svm._classes 
scikit-learn version: 1.5.2


In [20]:
# Display best hyperparameters
best_params = study.best_params
print("Best Hyperparameters:", best_params)
print("Best validation R² Score from Optuna:", study.best_value)

Best Hyperparameters: {'C': 1.0634704381649347, 'epsilon': 0.4153702019943078, 'gamma': 0.09626355619116081, 'kernel': 'rbf'}
Best validation R² Score from Optuna: 0.434008500533141


In [21]:
# Predict on the full dataset
y_pred_full = best_model.predict(x)

# Calculate R² score for the full dataset
r2_full = r2_score(y, y_pred_full)
print(f'R² Score for the full dataset: {r2_full:.4f}')

R² Score for the full dataset: 0.9480


In [22]:
mse_full = mean_squared_error(y, y_pred_full)
print(f'Mean Squared Error for the full dataset: {mse_full:.4f}')

Mean Squared Error for the full dataset: 0.0520


In [23]:
rmse_full = np.sqrt(mean_squared_error(y, y_pred_full))
print(f'Root Mean Squared Error for the full dataset: {rmse_full:.4f}')

Root Mean Squared Error for the full dataset: 0.2280


In [30]:
aad_full = np.mean(np.abs(y.flatten() - y_pred_full))
print(f'Average Absolute Deviation (AAD) for the full dataset: {aad_full:.4f}')

Average Absolute Deviation (AAD) for the full dataset: 0.1385


In [31]:
# Calculate AIC for the full dataset
n = len(y)  # number of samples
k = x.shape[1] + 1  # number of features + intercept (SVR has no explicit intercept, but we add 1 for model complexity)
aic_full = n * np.log(mse_full) + 2 * k
print(f'AIC for the full dataset: {aic_full:.4f}')

AIC for the full dataset: -78.7082


In [26]:
y_pred_full

array([-0.70333355, -0.88041285, -1.02909617, -0.97704425,  1.12914415,
        0.50124649,  1.54072018,  1.25707492, -0.95341411, -1.32841968,
       -1.00744493, -1.16110438,  0.78446528, -0.11908196,  1.39621157,
        0.82332998,  0.19142577, -0.40003119, -0.32052125,  0.55176686,
       -1.98323181,  1.78702886,  0.08350568, -0.33944691,  0.31858086,
        0.31858086,  0.31858086,  0.31858086,  0.31858086,  0.31858086])

In [32]:
# Inverse transform the scaled features and yields
features_unscaled = scaler_x.inverse_transform(x)
actual_yield_unscaled = scaler_y.inverse_transform(y).flatten()
predicted_yield_unscaled = scaler_y.inverse_transform(y_pred_full.reshape(-1, 1)).flatten()

# Create the dataframe
df_full_unscaled = pd.DataFrame(
    features_unscaled,
    columns=['Temperature', 'Catalyst dosage', 'MeOH-Oil ratio', 'Time']
)
df_full_unscaled['Actual Biodiesel Yield'] = actual_yield_unscaled
df_full_unscaled['Predicted Biodiesel Yield'] = predicted_yield_unscaled

df_full_unscaled

,Temperature,Catalyst dosage,MeOH-Oil ratio,Time,Actual Biodiesel Yield,Predicted Biodiesel Yield
0,60.0,2.0,12.0,1.38,47.0264,46.497708
1,80.0,2.0,12.0,1.38,43.5159,42.987157
2,60.0,4.0,12.0,1.38,33.2807,40.039549
3,80.0,4.0,12.0,1.38,40.5441,41.071464
4,60.0,2.0,24.0,1.38,84.0970,82.826103
5,80.0,2.0,24.0,1.38,69.8490,70.378195
6,60.0,4.0,24.0,1.38,91.5153,90.985491
7,80.0,4.0,24.0,1.38,88.0412,85.362298
8,60.0,2.0,12.0,3.13,42.5897,41.539926
9,80.0,2.0,12.0,3.13,28.7631,34.105538
